# FEM Simulatons of Stationary Stokes Flow on Tank with Nozzle Geometry      

##  Import Packages

In [1]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

More later.

## Section 2: Read 2D Mesh from File and Perform Elementary Checks

In [2]:
grid  = togrid("tankAndNozzle.msh")
inlet = getfacetset(grid, "inlet")
wall  = getfacetset(grid, "wall");

Info    : Reading 'tankAndNozzle.msh'...
Info    : 17 entities
Info    : 14713 nodes
Info    : 29424 elements
Info    : Done reading 'tankAndNozzle.msh'


In [3]:
getfacetset(grid,"wall")
getfacetset(grid,"inlet");

## Section 3: Functions for Assembly of Stiffness Matrix and Load Vector 
Note: wish to modified Ferrire tutorial code by removing the volumetric source term. 

In [4]:
function assemble_system!(K, f, dh, cvu, cvp,viscosity)
    assembler = start_assemble(K, f)
    ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    fe = zeros(ndofs_per_cell(dh))
    range_u = dof_range(dh, :u)
    ndofs_u = length(range_u)
    range_p = dof_range(dh, :p)
    ndofs_p = length(range_p)
    ϕᵤ = Vector{Vec{2,Float64}}(undef, ndofs_u)
    ∇ϕᵤ = Vector{Tensor{2,2,Float64,4}}(undef, ndofs_u) # 3-by-3 tensor 
    divϕᵤ = Vector{Float64}(undef, ndofs_u)
    ϕₚ = Vector{Float64}(undef, ndofs_p)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        ke .= 0
        fe .= 0
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            for i in 1:ndofs_u
                ϕᵤ[i] = shape_value(cvu, qp, i)
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end
            for i in 1:ndofs_p
                ϕₚ[i] = shape_value(cvp, qp, i)
            end
            # u-u
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += viscosity*( ∇ϕᵤ[i] ⊡ ∇ϕᵤ[j] ) * dΩ
            end
            # u-p
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] += ( -divϕᵤ[i] * ϕₚ[j] ) * dΩ
            end
            # p-u
            for (i, I) in pairs(range_p), (j, J) in pairs(range_u)
                ke[I, J] += ( -divϕᵤ[j] * ϕₚ[i] ) * dΩ
            end
            # rhs
            for (i, I) in pairs(range_u)
                x = spatial_coordinate(cvu, qp, getcoordinates(cell))
                # b = exp(-100 * norm(x - Vec{2}((0.75, 0.1)))^2)
                bv = Vec{2}((0.0, 0.0))
                fe[I] += (ϕᵤ[i] ⋅ bv) * dΩ
            end
        end
        assemble!(assembler, celldofs(cell), ke, fe)
    end
    return K, f
end

function setup_mean_constraint(dh, fvp)
    assembler = Ferrite.COOAssembler()
    # All external boundaries
    set = union(
            getfacetset(dh.grid, "wall"),
            getfacetset(dh.grid, "inlet"),
    )
    # Allocate buffers
    range_p = dof_range(dh, :p)
    element_dofs = zeros(Int, ndofs_per_cell(dh))
    element_dofs_p = view(element_dofs, range_p)
    element_coords = zeros(Vec{2}, 3) # assuming 2D mesh with triangles only 
    Ce = zeros(1, length(range_p)) # Local constraint matrix (only 1 row)
    # Loop over all the boundaries
    for (ci, fi) in set
        Ce .= 0
        getcoordinates!(element_coords, dh.grid, ci)
        Ferrite.reinit!(fvp, element_coords, fi)
        celldofs!(element_dofs, dh, ci)
        for qp in 1:getnquadpoints(fvp)
            dΓ = getdetJdV(fvp, qp)
            for i in 1:getnbasefunctions(fvp)
                Ce[1, i] += shape_value(fvp, qp, i) * dΓ
            end
        end
        # Assemble to row 1
        assemble!(assembler, [1], element_dofs_p, Ce)
    end
    C, _ = finish_assemble(assembler)
    # Create an AffineConstraint from the C-matrix
    _, J, V = findnz(C)
    _, constrained_dof_idx = findmax(abs2, V)
    constrained_dof = J[constrained_dof_idx]
    V ./= V[constrained_dof_idx]
    mean_value_constraint = AffineConstraint(
        constrained_dof,
        Pair{Int,Float64}[J[i] => -V[i] for i in 1:length(J) if J[i] != constrained_dof],
        0.0,
    )

    return mean_value_constraint
end

setup_mean_constraint (generic function with 1 method)

## Section 4: Set-up, Assembly and Solve

In [5]:
dim = 2 
degree = 1

# Interpolations
ipu = Lagrange{RefTriangle,degree+1}() ^ dim # quadratic for 2 velocity components 
ipp = Lagrange{RefTriangle,degree}()         # linear for scalar pressure 

# FE values
qr = QuadratureRule{RefTriangle}(2*degree+1)
ipg = Lagrange{RefTriangle,1}() # linear geometric interpolation
cvu = CellValues(qr, ipu, ipg) # observe three arguments - need to document whether this is required 
cvp = CellValues(qr, ipp, ipg) # observe three arguments - need to document whether this is required
qr_facet = FacetQuadratureRule{RefTriangle}(2)
fvp = FacetValues(qr_facet, ipp, ipg) # facet values for pressure  

# Dofs
dh = DofHandler(grid)
add!(dh, :u, ipu)
add!(dh, :p, ipp)
close!(dh) 

# Boundary conditions 
ch = ConstraintHandler(dh)

vmax = 10. 
Rin = 0.02 
parabolic_inflow_profile(x) = Vec((4*vmax*(Rin+x[2])*(Rin-x[2]), 0.0))

# Boundary conditions part (1/3): Dirichlet BC for the velocity at the top lid 
inlet = getfacetset(dh.grid, "inlet")
dbc1 = Dirichlet(:u, inlet, x ->  parabolic_inflow_profile(x) )
add!(ch, dbc1)

# Boundary conditions part (2/3): no slip boundary condition - impose velocity to be zero vector on the walls   
wall = union(
    getfacetset(grid, "wall"),
)
dbc2 = Dirichlet(:u, wall, (x, t) -> [0, 0])
add!(ch, dbc2)
    
# Boundary conditions part (3/3): apply pressure constraint
mean_value_constraint = setup_mean_constraint(dh, fvp)
add!(ch, mean_value_constraint)
#dbc3 = Dirichlet(:p, inlet, (x, t) -> 0)
#add!(ch, dbc3)
    
# Finalize
close!(ch)

 # Global tangent matrix and rhs
coupling = [true true; true false] # no coupling between pressure test/trial functions
K = allocate_matrix(dh, ch; coupling=coupling)
f = zeros(ndofs(dh))

# Assemble system
viscosity = 1e3
assemble_system!(K, f, dh, cvu, cvp, viscosity); 

# Apply boundary conditions and solve
apply!(K, f, ch)
u = K \ f;

VTKGridFile("stokes_2d_tankNozzle", dh) do vtk
    write_solution(vtk, dh, u)
    Ferrite.write_constraints(vtk, ch)
end

VTKGridFile for the closed file "stokes_2d_tankNozzle.vtu".

## Section 5: Transport of Passive Tracer

In [56]:
function assemble_flow_system!(K, dh, cvu, cvp,viscosity)
    assembler = start_assemble(K)
    range_u = dof_range(dh, :u)
    ndofs_u = length(range_u)
    range_p = dof_range(dh, :p)
    ndofs_p = length(range_p)
    ndofs_up = ndofs_u + ndofs_p 
    range_up = [range_u; range_p]
    ke = zeros(ndofs_up, ndofs_up)
    ϕᵤ = Vector{Vec{2,Float64}}(undef, ndofs_u)
    ∇ϕᵤ = Vector{Tensor{2,2,Float64,4}}(undef, ndofs_u) # 2-by-2 tensor 
    divϕᵤ = Vector{Float64}(undef, ndofs_u)
    ϕₚ = Vector{Float64}(undef, ndofs_p)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        ke .= 0
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            for i in 1:ndofs_u
                ϕᵤ[i] = shape_value(cvu, qp, i)
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end
            for i in 1:ndofs_p
                ϕₚ[i] = shape_value(cvp, qp, i)
            end
            # u-u
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += viscosity*( ∇ϕᵤ[i] ⊡ ∇ϕᵤ[j] ) * dΩ
            end
            # u-p
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] += ( -divϕᵤ[i] * ϕₚ[j] ) * dΩ
            end
            # p-u
            for (i, I) in pairs(range_p), (j, J) in pairs(range_u)
                ke[I, J] += ( -divϕᵤ[j] * ϕₚ[i] ) * dΩ
            end
        end
        assemble!(assembler, celldofs(cell), ke)
    end
    return K
end

assemble_flow_system! (generic function with 1 method)

In [6]:
function assemble_chemistry_system!(K, dh_u12, cellvalues_u1, cellvalues_u2, dh_up, cvu, pres_vel)
    assembler = start_assemble(K)
    range_u1  = dof_range(dh_u12, :u1)
    ndofs_u1  = length(range_u1)
    range_u2  = dof_range(dh_u12, :u2)
    ndofs_u2  = length(range_u2)
    ndofs_u12 = ndofs_u1 + ndofs_u2  
    range_u12 = [range_u1; range_u2]
    Ke = zeros(ndofs_u12, ndofs_u12)
    u1  = Vector{Float64}(undef, ndofs_u1)
    ∇u1 = Vector{Vec{2,Float64}}(undef, ndofs_u1)
    u2  = Vector{Float64}(undef, ndofs_u2)
    ∇u2 = Vector{Vec{2,Float64}}(undef, ndofs_u2)
    range_u = dof_range(dh_up, :u)
    
    for (cell_num, cell) in enumerate(CellIterator(dh_u12))

        fill!(Ke, 0)

        Ferrite.reinit!(cellvalues_u1, cell)

        vel_dof_loc = pres_vel[celldofs(dh_up,cell_num)[range_u]]
        
        for qp in 1:getnquadpoints(cellvalues_u1)
            dΩ = getdetJdV(cellvalues_u1, qp)
            for i in 1:ndofs_u1
                u1[i] = shape_value(cellvalues_u1, qp, i)
                ∇u1[i] = shape_gradient(cellvalues_u1, qp, i)
            end
            # u2-u2
            vel_loc = function_value(cvu, qp, vel_dof_loc)
            # velocity_loc = [1,0]
            for (i, I) in pairs(range_u2), (j, J) in pairs(range_u2) 
                Ke[I, J] += ( ∇u1[i] ⋅ ∇u1[j] - (vel_loc ⋅ ∇u1[i] )* u1[j]) * dΩ
            end
        end 
            
        assemble!(assembler, celldofs(cell), Ke)
    end
    return K     
end

assemble_chemistry_system! (generic function with 1 method)

In [91]:
dim = 2 
degree = 1

# Interpolations
ipu = Lagrange{RefTriangle,degree+1}() ^ dim # quadratic for 2 velocity components 
ipp = Lagrange{RefTriangle,degree}()         # linear for scalar pressure 
ipu1 = Lagrange{RefTriangle,degree+1}()      # linear for scalar pressure 
ipu2 = Lagrange{RefTriangle,degree+1}()      # linear for scalar pressure 

# FE values
qr = QuadratureRule{RefTriangle}(2*degree+1)
ipg = Lagrange{RefTriangle,1}() # linear geometric interpolation
cvu = CellValues(qr, ipu, ipg) # observe three arguments - need to document whether this is required 
cvp = CellValues(qr, ipp, ipg) # observe three arguments - need to document whether this is required
cvu1 = CellValues(qr, ipu1, ipg) 
cvu2 = CellValues(qr, ipu2, ipg)
qr_facet = FacetQuadratureRule{RefTriangle}(2)
fvp = FacetValues(qr_facet, ipp, ipg) # required for pressure constraint 

# Dofs for pressure - velocity
dh_up = DofHandler(grid)
add!(dh_up, :u, ipu)
add!(dh_up, :p, ipp)
close!(dh_up) 

# Dofs for chemistry 
dh_u12 = DofHandler(grid)
add!(dh_u12, :u1, ipu1)
add!(dh_u12, :u2, ipu2)
close!(dh_u12)

# Boundary conditions for pressure - velocity
ch_up = ConstraintHandler(dh_up)

vmax = 10. 
Rin = 0.02 
parabolic_inflow_profile(x) = Vec((4*vmax*(Rin+x[2])*(Rin-x[2]), 0.0))

# Boundary conditions part (1/3): Dirichlet BC for the velocity at the top lid 
dbc1 = Dirichlet(:u, inlet, x ->  parabolic_inflow_profile(x) )
add!(ch_up, dbc1)

# Boundary conditions part (2/3): no slip boundary condition - impose velocity to be zero vector on the walls   
dbc2 = Dirichlet(:u, wall, (x, t) -> [0, 0])
add!(ch_up, dbc2)
    
# Boundary conditions part (3/3): apply pressure constraint
mean_value_constraint = setup_mean_constraint(dh_up, fvp)
add!(ch_up, mean_value_constraint)
#dbc3 = Dirichlet(:p, inlet, (x, t) -> 0)
#add!(ch, dbc3)

# Finalize
close!(ch_up)

# Boundary conditions for pressure - velocity
ch_u12 = ConstraintHandler(dh_u12)

rho_saturated = .5
dbc4 = Dirichlet(:u1, inlet, x -> rho_saturated)
add!(ch_u12, dbc4)

dbc5 = Dirichlet(:u2, inlet, x -> rho_saturated)
add!(ch_u12, dbc5)

# Finalize
close!(ch_u12)

dof_range_u1 = dof_range(dh_u12, :u1)
dof_range_u2 = dof_range(dh_u12, :u2)
range_u1 =  unique!(reduce(vcat, [ celldofs(dh_u12, ce)[dof_range_u1] for ce in 1:length(grid.cells) ]))
range_u2 =  unique!(reduce(vcat, [ celldofs(dh_u12, ce)[dof_range_u2] for ce in 1:length(grid.cells) ]))

# Global tangent matrix and rhs for pressure - velocity 
coupling = [true true; true false] 
K_up = allocate_matrix(dh_up, ch_up; coupling=coupling)
f_up = zeros(ndofs(dh_up)); 

# Global tangent matrix and rhs for chemistry 
coupling = [true false; false true] 
K_u12 = allocate_matrix(dh_u12, ch_u12; coupling=coupling)
f_u12 = zeros(ndofs(dh_u12));

In [92]:
# Assemble system
viscosity = 1e3
assemble_flow_system!(K_up, dh_up, cvu, cvp, viscosity) 

# Apply boundary conditions and solve
apply!(K_up, f_up, ch_up)

u_up = K_up \ f_up  

assemble_chemistry_system!(K_u12, dh_u12, cvu1, cvu2, dh_up, cvu, u_up)
K_u12[range_u1, range_u1] .= I(length(range_u1)) 
f_u12[range_u1] .= rho_saturated 
f_u12[range_u2] .= 0.
apply!(K_u12, f_u12, ch_u12)

u_u12 = (K_u12) \ f_u12 

VTKGridFile("stokes_2d_channel", dh_up) do vtk
    write_solution(vtk, dh_up, u_up)
    write_solution(vtk, dh_u12, u_u12)
    # Ferrite.write_constraints(vtk, ch)
end

VTKGridFile for the closed file "stokes_2d_channel.vtu".